In [40]:
from pyomo.environ import *
import pandas as pd
import highspy

## Declare the Variables

In [41]:
Sector = ['Pertanian', 'Manufaktur', 'Jasa', 'Infrastruktur']   # Sector names
S = ['A', 'M', 'S', 'I']                                   # Sector codes

In [42]:
d = {'A': 50*1.6, 'M': 75*1.6, 'S': 50*1.6, 'I': 40*1.6}        # Minimum demand
va = {'A': 0.6, 'M': 0.8, 'S': 0.75, 'I': 0.9}  # Value added coefficients

In [43]:
# Resource usage coefficients
labour = {'A': 2, 'M': 3, 'S': 1, 'I': 3}
capital = {'A': 1.1, 'M': 2, 'S': 0.8, 'I': 2.5}
environment = {'A': 0.5, 'M': 0.9, 'S': 0.3, 'I': 1.1}

# Resource availability
L_max = 1000
C_max = 1200 
E_max = 400

## Declare Model

In [44]:
def build_model(L_cap, E_cap):
    m = ConcreteModel()
    m.I = Set(initialize=S)
    m.p = Var(m.I, domain=NonNegativeReals)
    m.y = Var(m.I, domain=Binary)

    # Demand naik 60%
    d2 = {i: 1.6 * d[i] for i in d}

    # Resource constraints
    m.labour = Constraint(expr = sum(labour[i] * m.p[i] for i in m.I) <= L_cap)
    m.capital = Constraint(expr = sum(capital[i] * m.p[i] for i in m.I) <= C_max)
    m.environment = Constraint(expr = sum(environment[i] * m.p[i] for i in m.I) <= E_cap)
    m.fiscal = Constraint(expr = m.p['I'] <= 120)

    # Big-M
    M_big = {'A':500, 'M':500, 'S':500, 'I':500}
    def link_rule(m, i):
        return m.p[i] <= M_big[i] * m.y[i]
    m.link = Constraint(m.I, rule=link_rule)

    # Min demand if active
    def min_demand_rule(m, i):
        return m.p[i] >= d2[i] * m.y[i]
    m.min_demand = Constraint(m.I, rule=min_demand_rule)

    # Objective
    m.obj = Objective(expr = sum(va[i] * m.p[i] for i in m.I), sense=maximize)

    return m


In [45]:
def summarize_model(model):
    # GDP per sector
    gdp = {code: va[code] * model.p[code]() for code in S}
    total_gdp = sum(gdp.values())

    # Resource usage
    labour_used   = sum(labour[i]     * model.p[i]() for i in model.I)
    capital_used  = sum(capital[i]    * model.p[i]() for i in model.I)
    emission_used = sum(environment[i]* model.p[i]() for i in model.I)

    # Active sectors (based on y[i])
    active_sectors = [name for code, name in zip(S, Sector) 
                      if model.y[code]() > 0.5]
    status_str = ", ".join(active_sectors) if active_sectors else "Tidak ada sektor aktif"

    return {
        "Pertanian": gdp['A'],
        "Manufaktur": gdp['M'],
        "Jasa": gdp['S'],
        "Infrastruktur": gdp['I'],
        "TOTAL": total_gdp,
        "Tenaga Kerja": labour_used,
        "Modal": capital_used,
        "Batas Emisi": emission_used,
        "Status": status_str
    }


In [46]:
solver = SolverFactory('highs')

# Scenario 1: Normal
m_normal = build_model(L_cap=1000, E_cap=400)
solver.solve(m_normal)
normal_data = summarize_model(m_normal)

# Scenario 2: Less Emission
m_lessE = build_model(L_cap=1000, E_cap=200)
solver.solve(m_lessE)
lessE_data = summarize_model(m_lessE)

# Scenario 3: Less Worker
m_lessL = build_model(L_cap=500, E_cap=400)
solver.solve(m_lessL)
lessL_data = summarize_model(m_lessL)

final_table = pd.DataFrame({
    "Normal": normal_data,
    "Less Emission": lessE_data,
    "Less Worker": lessL_data
})
final_table

,Normal,Less Emission,Less Worker
Pertanian,150.0,76.8,0.0
Manufaktur,0.0,0.0,0.0
Jasa,375.0,340.0,375.0
Infrastruktur,0.0,0.0,0.0
TOTAL,525.0,416.8,375.0
Tenaga Kerja,1000.0,709.333333,500.0
Modal,675.0,503.466667,400.0
Batas Emisi,275.0,200.0,150.0
Status,"Pertanian, Jasa","Pertanian, Jasa",Jasa


## Conclusion

1. Sektor Pertanian dan Jasa tetap dijalankan
2. Jasa berkontribusi 375(70%) dan Pertanian 150(30%)
3. Pada model LP di Soal 1, seluruh sektor wajib berproduksi sehingga alokasi output lebih tersebar. Tidak ada fleksibilitas untuk mematikan sektor yang kurang efisien. Namun dalam model MILP ini, sektor boleh dimatikan. Ketika diberikan kebebasan tersebut, solusi optimal berubah drastis, ekonomi lebih terfokus pada sektor yang memberikan nilai tambah paling tinggi per unit sumber daya. Akibatnya:
   - Jasa menjadi sektor inti di semua skenario,
   - Pertanian hanya aktif ketika tenaga kerja masih mencukupi,
   - Manufaktur dan Infrastruktur tidak aktif di semua kondisi karena rasio input–output mereka tidak kompetitif.
4. Dampak batas emisi lebih ketat (E=200)
   - Output sektor intensitas emisi rendah (Jasa, Pertanian) turun.
   - Sektor beremisi tinggi tetap tidak aktif.
   - total PDB turun dari 525 → 416.8.
   - Artinya: ada trade-off antara pertumbuhan ekonomi dan lingkungan.
5. Dampak batas tenaga kerja rendah (L=500)
   - Hanya sektor Jasa yang bertahan → paling hemat tenaga kerja.
   - Pertanian mati karena padat karya.
   - PDB turun signifikan (525 → 375).
   - Artinya: tenaga kerja adalah bottleneck paling penting dalam sistem ini.